In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [ ]:
#dataset link
# https://archive.ics.uci.edu/dataset/53/iris
# we will use this dataset from sklearn
#https://scikit-learn.org/dev/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris

In [12]:
iris=load_iris()
df=pd.DataFrame(iris.data,columns=iris.feature_names)
df['target']=iris.target


In [13]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [14]:
train_df,test_df=train_test_split(df,test_size=0.2,random_state=42,stratify=df['target'])

In [16]:
x_train=train_df.drop(columns=['target'])
y_train=train_df['target']
x_test=test_df.drop(columns=['target'])
y_test=test_df['target']

In [17]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [28]:
x_train_tensor=torch.tensor(x_train,dtype=torch.float32)
y_train_tensor=torch.tensor(y_train.values,dtype=torch.long)
x_test_tensor=torch.tensor(x_test,dtype=torch.float32)
y_test_tensor=torch.tensor(y_test.values,dtype=torch.long)

In [29]:
class IrisClassifier(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim):
        super(IrisClassifier,self).__init__()
        self.network=nn.Sequential(
            nn.Linear(input_dim,hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,output_dim)
        )
    def forward(self,x):
        return self.network(x)
        

In [30]:
input_dim=x_train.shape[1]
hidden_dim=16
output_dim=3
model=IrisClassifier(input_dim,hidden_dim,output_dim)
critirion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.01)

In [31]:
# training the model
epochs=500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    predictions=model(x_train_tensor)
    loss=critirion(predictions,y_train_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1)%50 ==0:
        print(f' epoch [{epoch+1}]/{epochs}, loss: {loss.item():.4f}')

 epoch [50]/500, loss: 0.0782
 epoch [100]/500, loss: 0.0271
 epoch [150]/500, loss: 0.0108
 epoch [200]/500, loss: 0.0018
 epoch [250]/500, loss: 0.0007
 epoch [300]/500, loss: 0.0004
 epoch [350]/500, loss: 0.0003
 epoch [400]/500, loss: 0.0002
 epoch [450]/500, loss: 0.0001
 epoch [500]/500, loss: 0.0001


In [32]:
model.eval()
with torch.no_grad():
    y_pred=model(x_test_tensor)
    y_pred_labels=torch.argmax(y_pred,dim=1)
    accuracy=(y_pred_labels==y_test_tensor).sum().item()/y_test_tensor.size(0)
    print(f'test acurracy: {accuracy:.4f}')


test acurracy: 0.9667


In [33]:
def predict_iris(sepal_length,sepal_width,petal_length,petal_width):
    input_data=np.array([[sepal_length,sepal_width,petal_length,petal_width]])
    input_data=scaler.transform(input_data)
    input_tensor=torch.tensor(input_data,dtype=torch.float32)
    model.eval()
    with torch.no_grad():
        predection=model(input_tensor)
        predection_class=torch.argmax(predection,dim=1).item()
    return iris.target_names[predection_class]

In [35]:
print(predict_iris(5.1,3.5,1.4,0.2))

setosa


c:\Users\ragha\anaconda3\envs\pytorch\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
